In [1]:
import os
import zipfile
import random
import tensorflow as tf
import keras 


import random
import sys 

print(tf.__version__)
print(sys.version)

2.8.0
3.10.4 (main, Mar 25 2022, 00:00:00) [GCC 11.2.1 20220127 (Red Hat 11.2.1-9)]


In [2]:
import numpy as np
import matplotlib.pyplot as plt


# use seaborn plotting defaults
import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.image as mpimg
from PIL import Image
from PIL import ImageFilter


In [24]:
with open('./resnet101_transfer_learning/tflite_res101_best', 'rb') as f:
    tflite_best_resnet_instance = f.read()
interpreter_resnet_best = tf.lite.Interpreter(model_content=tflite_best_resnet_instance)
interpreter_resnet_best.allocate_tensors()
resnet_input_index = interpreter_resnet_best.get_input_details()[0]['index']
resnet_output_index = interpreter_resnet_best.get_output_details()[0]['index']

In [43]:
with open('./dense121_transfer_learning/tflite_dense121_best', 'rb') as f:
    tflite_best_dense_instance = f.read()
interpreter_dense_best = tf.lite.Interpreter(model_content=tflite_best_dense_instance)
interpreter_dense_best.allocate_tensors()
dense_input_index = interpreter_dense_best.get_input_details()[0]['index']
dense_output_index = interpreter_dense_best.get_output_details()[0]['index']

In [45]:
with open('./custom_model/tflite_custom_best', 'rb') as f:
    tflite_best_custom_instance = f.read()
interpreter_custom_best = tf.lite.Interpreter(model_content=tflite_best_custom_instance)
interpreter_custom_best.allocate_tensors()
custom_input_index = interpreter_custom_best.get_input_details()[0]['index']
custom_output_index = interpreter_custom_best.get_output_details()[0]['index'] 

In [18]:
N_PATH = '/bigdata3/OCT2017/all_data/NORMAL/'
DRU_PATH = '/bigdata3/OCT2017/all_data/DRUSEN/'
DME_PATH = '/bigdata3/OCT2017/all_data/DME/'
C_PATH = '/bigdata3/OCT2017/all_data/CNV/'

In [19]:
normals = os.listdir(N_PATH)
drusen = os.listdir(DRU_PATH)
dme = os.listdir(DME_PATH)
cnv = os.listdir(C_PATH)

In [20]:
types = ['CNV', 'DME', 'DRUSEN', 'NORMAL'] 


In [36]:
def predict(base_dir, source_list, interpreter, input_index, output_index):
    image_path = os.path.join(base_dir, random.choice(source_list))
    print(f'testing image: {image_path}')
    ni = tf.io.read_file(image_path)
    ni = tf.io.decode_jpeg(ni, channels=3)
    fni = tf.image.resize(
            ni,
            (256, 256),
            # method=ResizeMethod.BILINEAR,
            preserve_aspect_ratio=False,
            antialias=False,
            name=None
        )
    fni = np.float32(fni.numpy()/255.)
    fni = fni.reshape((1, 256, 256, 3))
    interpreter.set_tensor(input_index, fni)
    interpreter.invoke()
    result = interpreter.get_tensor(output_index)
    print(result)
    amax = np.argmax(result)
    print(amax)
    percent = result[0][amax] * 100
    image_type = types[amax]
    print(f'type: {image_type} confidence: {percent:.2f}%')
                    

In [41]:
predict(C_PATH, cnv, interpreter_resnet_best, resnet_input_index, resnet_output_index)

testing image: /bigdata3/OCT2017/all_data/CNV/CNV-6294785-133.jpeg
[[9.9921298e-01 7.7910628e-04 5.8956255e-07 7.3073893e-06]]
0
type: CNV confidence: 99.92%


In [44]:
predict(DRU_PATH, drusen, interpreter_dense_best, dense_input_index, dense_output_index)

testing image: /bigdata3/OCT2017/all_data/DRUSEN/DRUSEN-3281144-11.jpeg
[[6.9978363e-03 3.0431426e-03 9.8990250e-01 5.6502427e-05]]
2
type: DRUSEN confidence: 98.99%


In [47]:
predict(DME_PATH, dme, interpreter_custom_best, custom_input_index, custom_output_index)

testing image: /bigdata3/OCT2017/all_data/DME/DME-7793366-1.jpeg
[[2.1688447e-04 9.9973983e-01 3.3325272e-05 1.0053717e-05]]
1
type: DME confidence: 99.97%
